# Desafio 2:  Validação Cruzada Para Seleção de Modelos

## Objetivos:

O objetivo desse desafio é compreender como funciona a validação cruzada e os benefícios em se utilizar essa técnica para selecionar modelos mais genéricos.


## Conceitos:

A validação cruzada é uma técnica de avaliação de um modelo em treinamento que permite estimar a capacidade de **generalização** do mesmo. A idéia por trás da técnica é validar qual seria o desempenho do modelo, treinado com uma parte dados, sobre o resto dos dados, nunca vistos antes pelo modelo. 

Essa técnica possibilita que se teste essa hipótese sem ser necessário ter uma base de testes extra, o que é muito útil quando se têm poucos dados. O diagrama abaixo resume a idéia central da técnica:


![Diagrama explicativo: Cross-Validation usando K-Fold com ${K = 10}$](images/k-fold-diagram.png)

A técnica mais comum, denominada `K-Fold`, consiste em particionar o dataset em $K$ grupos, treinar com $K-1$ grupos de dados e validar o modelo treinado sobre o grupo restante. Deve-se repetir esse passo $K$ vezes, para que todos os grupos sejam usados, e então calcular as métricas de avaliação como a média das métricas calculadas em cada iteração. Uma outra maneira de se fazer isso é armazenar a predição sobre todos os dados **quando em validação** e calcular as métricas de desempenho sobre essas predições.

O limite dessa técnica é o `Leave-One-Out`, caso é o extremo em que se particiona o dataset em $N$ grupos. Esse limite, quando possível, traz a informação mais acurada sobre a capacidade de generalização do modelo. Essa técnica tem como desvantagem a quantidade enorme de treinamentos realizados: um para cada elemento da base de treino.

___

# Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
from sklearn.cross_validation import KFold
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics.regression import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# Dataset:

## Carregando os dados

In [ ]:
from sklearn.datasets import load_boston
dataset = load_boston()

### Sobre o Dataset

In [ ]:
print(dataset["DESCR"])

In [ ]:
dataset.keys()

### Separação das Features e Variável Dependente

In [ ]:
x = pd.DataFrame(
    columns=dataset["feature_names"],
    data=dataset["data"]
)

In [ ]:
y = dataset["target"]

In [ ]:
x.shape

In [ ]:
x.head()

In [ ]:
y.shape

In [ ]:
y[:5]

## Separação de Treino e Teste

In [ ]:
# splitting 
train_index, test_index = train_test_split(
    x.index, 
    test_size=.3, 
    random_state=42
)

x_train = x.loc[train_index, :]
x_test = x.loc[test_index, :]
y_train = y[train_index]
y_test = y[test_index]

In [ ]:
x_train.shape

In [ ]:
x_test.shape

## Normalizando Features com Z-Score

Passo importante quando se treina modelos lineares, por eliminar importâncias artificialmente grandes para features contendo valores muito grandes.

In [ ]:
zscore = StandardScaler().fit(x_train)
x_train = zscore.transform(x_train)
x_test = zscore.transform(x_test)

# Problemas

## A) Seleção de Hiperparâmetros

O uso mais comum da técnica é a seleção dos `hiperparâmetros` que definem o modelo; 
diferente dos parâmetros do modelo, que são ajustados a cada iteração do treinamento, os hiperparâmetros são condições fixadas sob as quais o treinamento ocorre. Um exemplo de hiperparâmetro é o $alpha$, definido para treinamentos das regressões `Ridge` e `Lasso`.

Nessa seção, será feita a busca pelos melhores hiperparâmetros de treinamento usando a $ElasticNet$, que combina a regularização `L1` e `L2` em um modelo único. 

### Escolha dos Hiperparâmetros

Os hiperparâmetros a serem variados são:
* `alpha`: mesmo parâmetro visto em Regularização, define a intensidade da regularização no modelo.
* `l1_ratio `: define o tipo de regularização, como mostrado abaixo:
    * $l1\_ratio = 1$: o treinamento ocorre com `L1` apenas
    * $l1\_ratio = 0$: o treinamento ocorre com `L2` apenas
    * $0 \leq l1\_ratio \leq 1$: o treinamento ocorre com uma combinação de `L1` com `L2` na proporção dada.

#### Variação dos hiperparâmetros

In [ ]:
# hiperparâmetros
param_grid = {
    "alpha": np.logspace(-3, 4, 11),          # Por quê `logspace`?
    "l1_ratio": np.linspace(0.01, 1.0, 11),
    "max_iter": [100],
    "positive": [True]
}

##### Treinamento dos modelos

Usar o [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) para treinar um conjunto de modelos com a combinação de todos os hiperparâmetros.

Parâmetros a serem utilizados:

* `estimator`: [`ElasticNet()`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)
* `param_grid `: `param_grid`
* `scoring`: ['neg_mean_squared_error', 'r2']
* `cv`: usar [`KFold(n_splits=10, shuffle=True, random_state=42)`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)
* `random_state`: 42

In [ ]:
""" Escreva a solução aqui """

####  Avaliação dos modelos treinados

Mostrar uma tabela contendo todos os treinamentos e suas avaliações e escolher um range de exploração onde o modelo consegue ter uma performance melhor.

Dica: usar a função [`heatmap`](https://seaborn.pydata.org/generated/seaborn.heatmap.html) do `seaborn` para visualizar os melhores parâmetros.

In [ ]:
""" Escreva a solução aqui """

## B) Estudo da influência do parâmetro $K$ sobre a generalização

Quanto mais `folds` forem usados na validação cruzada, mais realista será a estimativa do poder de generalização do modelo treinado. Como já foi citado, o uso de `LeaveOneOut` é o mais próximo de trazer essa estimativa, com o contraponto de ser o mais pesado computacionalmente.

Nessa seção, será feita uma comparação entre o desempenho dos modelos nas massas de validação (calculado com dados de treino durante a validação cruzada) e de teste. 

###### Importante:

Os hipeparâmetros a serem variados nos treinamentos devem ser obtidos na última etapa do item **A**.

####  Hiperparâmetros

Preencher `params_grid` com as faixas de hiperparâmetros de melhor performance.

In [ ]:
""" Complete os espaços com ? """
param_grid = {
    "alpha": ?,
    "l1_ratio": ?,
    "max_iter": [100],
    "positive": [True]
}

####  Treinar os modelos para os valores de $K$

Treinar os modelos para cada valor de $K$ em `k_list`. Armazenar, para cada $K$, os valores de $R^2$ e de $MSE$ calculados sobre a **massa de testes**.

In [ ]:
""" Escreva a solução aqui """

####  Avaliação da generalização

Mostrar em dois plots (um para $R^2$ e outro para $MSE$) a comparação entre as métricas de validação (do cross-validation) e as medidas sobre a massa de teste. 

* Eixo X: valores de  $K$
* Eixo Y: métricas de avaliação ($R^2$ / $MSE$)


In [ ]:
k_values = [3, 5, 10, 20, 35, 50, 70, 100]

In [ ]:
""" Escreva a solução aqui """